Pre Trained Model in Deep Learning
In transfer Learning we import a Model which is already Trained Like Resnet50 in this we will have
Convolution Layer + Feed Forward Layer + Dense.
In this we fix the weights of Convolution Layer and we train only just Dense Layer. For our specific Dataset. In Fine-Tuning we take some Convolution Layer + Dense Layer for training.

In [1]:
import numpy as np
import keras
from keras import layers
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [3]:
!kaggle datasets download -d salader/dogs-vs-cats

 98% 1.05G/1.06G [00:09<00:00, 95.8MB/s]
100% 1.06G/1.06G [00:09<00:00, 121MB/s] 


In [4]:
import zipfile
zip_ref = zipfile.ZipFile('/content/dogs-vs-cats.zip','r')
zip_ref.extractall('/content')
zip_ref.close()

In [5]:
from keras import Sequential
from keras.layers import Flatten,Dense
from keras.applications.vgg16 import VGG16

We have Loaded VGG16 Model which is trained on imagenet Dataset. And include_top = False ( Imply We will fix all the Convolution Layer And we Just edit Dense Layer )

In [6]:
conv_base = VGG16(
    weights = 'imagenet',
    include_top = False,
    input_shape = (256,256,3)
)

58889256/58889256 [==============================] - 0s 0us/step


In [7]:
model = Sequential()

model.add(conv_base)
model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 8, 8, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 32768)             0         
                                                                 
 dense (Dense)               (None, 256)               8388864   
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 23103809 (88.13 MB)
Trainable params: 23103809 (88.13 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [9]:
conv_base.trainable = False

In [10]:
# Generator
train_ds = keras.utils.image_dataset_from_directory(
    directory = '/content/train',
    labels = 'inferred',
    label_mode='int', # It will asign 0-> to Dog and 1 to-> Cat
    batch_size=32,
    image_size=(256,256)  # We are doing so inordere to make the image of all image to same level
)

validation_ds = keras.utils.image_dataset_from_directory(
    directory = '/content/test',
    labels = 'inferred',
    label_mode='int', # It will asign 0-> to Dog and 1 to-> Cat
    batch_size=32,
    image_size=(256,256)  # We are doing so inordere to make the image of all image to same level
)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.


In [11]:
import tensorflow as tf

In [12]:
def process(image,label):
  image = tf.cast(image/255. ,tf.float32)
  return image,label
train_ds = train_ds.map(process)
validation_ds = validation_ds.map(process)

In [13]:
model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

In [14]:
history = model.fit(train_ds,epochs=10,validation_data=(validation_ds))

Epoch 1/10
625/625 [==============================] - 144s 212ms/step - loss: 0.2545 - accuracy: 0.9027 - val_loss: 0.1498 - val_accuracy: 0.9368
Epoch 2/10
625/625 [==============================] - 135s 216ms/step - loss: 0.1427 - accuracy: 0.9428 - val_loss: 0.1584 - val_accuracy: 0.9334
Epoch 3/10
625/625 [==============================] - 148s 236ms/step - loss: 0.0844 - accuracy: 0.9664 - val_loss: 0.1573 - val_accuracy: 0.9368
Epoch 4/10
625/625 [==============================] - 149s 237ms/step - loss: 0.0577 - accuracy: 0.9789 - val_loss: 0.1793 - val_accuracy: 0.9344
Epoch 5/10
625/625 [==============================] - 150s 240ms/step - loss: 0.0570 - accuracy: 0.9775 - val_loss: 0.2286 - val_accuracy: 0.9306
Epoch 6/10
625/625 [==============================] - 148s 237ms/step - loss: 0.0395 - accuracy: 0.9843 - val_loss: 0.4867 - val_accuracy: 0.8880
Epoch 7/10
625/625 [==============================] - 134s 214ms/step - loss: 0.0307 - accuracy: 0.9884 - val_loss: 0.2388 -